<a href="https://colab.research.google.com/github/anvichip/document-image-talker/blob/main/document_talker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [1]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 2.4 MB/s eta 0:00:00


In [3]:
!pip install opencv-python
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils
!apt update
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip3 install opencv-contrib-python==4.5.5.62
!pip install --upgrade pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (628 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compu

In [4]:
!pip install transformers

In [5]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.39-cp310-cp310-manylinux_2_35_x86_64.whl size=15188356 sha256=efbc548b4d58c36e7a3d512e0cf9709d548775aad368c0964ff3b7c8880467db
  Stored in directory: /root/.cache/pip/wheels/23/ee/3c/4a12c19a605d827cf3f67aa4a7d053784959232a964580f626
Successfully built llama-cpp-python


### Text Extraction from Image

In [6]:
import cv2
import pytesseract
from pdf2image import convert_from_path
from google.colab.patches import cv2_imshow
import os
import numpy as np

In [7]:
def extract_from_image(path):
  import os
  image = cv2.imread(path)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)
  # Adaptive thresholding
  thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)
  # Adjust contrast
  contrast = 0.95  # Adjust the value as desired (between 0 and 1)
  adjusted_image = np.clip(thresh.astype(np.float32) * contrast, 0, 255).astype(np.uint8)
  text = str(pytesseract.image_to_string(adjusted_image, config='--psm 6'))
  # Dilation
  #kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  #dilated = cv2.dilate(thresh, kernel, iterations=1)
  # Display the enhanced image
  #cv2_imshow(adjusted_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
  # image_p
  # os.mkdir()
  with open("image_text.txt","w+") as f:
    f.write(text)
  return os.path.abspath("image_text.txt")


### Making Llama2 Query Engine

In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9693238438673467031
 xla_global_id: -1]

In [8]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

In [9]:
model_url = 'https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf'

In [11]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=1500,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

RuntimeError: Failed to load shared library '/usr/local/lib/python3.10/dist-packages/llama_cpp/libllama.so': libcuda.so.1: cannot open shared object file: No such file or directory

In [ ]:
# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
# create a service context
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

In [36]:
# #function to save file in colab in folder documents
# def save_file():
#   # Directory
#   directory = "document"

#   # Parent Directory path
#   parent_dir = "/content/documents"

#   # Path
#   path = os.path.join(parent_dir, directory)
#   os.mkdir(path)

In [37]:
#function which checks the extension of file
def check_ext(path):
  f = os.listdir(path)[0]
  if f.endswith('.pdf'):
    document = SimpleDirectoryReader(path).load_data()
  elif f.endswith('.jpg') or path.endswith('png'):
    txt_path = extract_from_image(f)
    document = SimpleDirectoryReader(txt_path).load_data()
  else:
    return None
  return document


In [ ]:
print(check_ext('/content/documents'))

In [8]:
#function to pass query to llama2
def get_llama_response(document, ques):
  index = VectorStoreIndex.from_documents(document, service_context=service_context)
  query_engine = index.as_query_engine()
  response = query_engine.query(ques)
  return response

In [ ]:
def main():
  path = input("Enter the Document/Image you wish to query ")
  doc = check_ext(path)
  ques = input("Enter the Document/Image you wish to query ")
  response = get_response(doc,ques)
  print(response)

In [ ]:
main()

Gradio

In [ ]:
!pip install gradio

In [2]:
import gradio as gr

In [14]:
def upload_file(files):
    file_paths = [file.name for file in files]
    return file_paths

In [4]:
import gradio as gr
import os
import shutil
def save_pdf(files):
  print (files)
    # Define the path where you want to save the files
  if not os.path.exists('documents'):
      os.makedirs('documents')

  # Loop over the list of files
  for i, file in enumerate(files):
      path = f"/content/documents/uploaded_file_{i}.pdf"
      shutil.copyfile(file, path)

      # # Write the file to disk
      # with open(path, "wb") as f:
      #     f.write(file.value.encode())

      print(f"File {i} saved")

def get_response(prompt,file):
  save_pdf(file)
  response = 'pk'
  response = get_llama_response()
  return response


iface = gr.Interface(
    fn=get_response,
    inputs=["text",gr.File(file_count = 'multiple')],
    outputs="text"
)

iface.launch(debug = True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0461be0a7796e2b08d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


['/tmp/gradio/e6ce4aa32eb3acb0648dcbddee34969c73871c80/RAG-Document 1.pdf']
File 0 saved
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0461be0a7796e2b08d.gradio.live


In [10]:
import os
import shutil
os.listdir('sample_data')

['README.md',
 'anscombe.json',
 'mnist_train_small.csv',
 'california_housing_test.csv',
 'mnist_test.csv',
 'california_housing_train.csv']

In [12]:
# files = ['/content/sample_data/README.md',
# '/content/sample_data/anscombe.json',
# '/content/sample_data/mnist_train_small.csv',
# '/content/sample_data/california_housing_test.csv',
# '/content/sample_data/mnist_test.csv',
# '/content/sample_data/california_housing_train.csv']
for i, file in enumerate(files):
  path = '/content/documents' + str(i)
  shutil.copyfile(file, path)